# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import nltk
nltk.download(['punkt', 'wordnet'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///category_message.db')
df = pd.read_sql_table('category_message', engine)

X = df['message']
Y = df.iloc[:, 4:]

display(Y.head(10))



,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#set ML pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
#split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

#train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)
col_names = list(Y.columns.values)

for i in range(len(col_names)):
    print("Precision, Recall, F1 score for {}".format(Y_test.columns[i]))
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))


Precision, Recall, F1 score for related
             precision    recall  f1-score   support

          0       0.58      0.37      0.45      1458
          1       0.83      0.92      0.87      5024
          2       0.69      0.19      0.30        47

avg / total       0.77      0.79      0.77      6529

Precision, Recall, F1 score for request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5388
          1       0.83      0.38      0.52      1141

avg / total       0.87      0.88      0.86      6529

Precision, Recall, F1 score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6500
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6529

Precision, Recall, F1 score for aid_related
             precision    recall  f1-score   support

          0       0.71      0.89      0.79      3780
          1       0.76      0.50 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6450
          1       0.00      0.00      0.00        79

avg / total       0.98      0.99      0.98      6529

Precision, Recall, F1 score for other_infrastructure
             precision    recall  f1-score   support

          0       0.95      1.00      0.98      6232
          1       0.00      0.00      0.00       297

avg / total       0.91      0.95      0.93      6529

Precision, Recall, F1 score for weather_related
             precision    recall  f1-score   support

          0       0.86      0.96      0.90      4772
          1       0.83      0.57      0.67      1757

avg / total       0.85      0.85      0.84      6529

Precision, Recall, F1 score for floods
             precision    recall  f1-score   support

          0       0.93      1.00      0.96      5988
          1       0.85      0.22      0.35       541

avg / total       0.93      0.93      0.91      6529



### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.75, 1.0),
    'tfidf__use_idf': (True, False)
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [66]:
cv.fit(X_train, Y_train)

Y_pred = cv.predict(X_test)

for i in range(len(col_names)):
    print("Precision, Recall, F1 score for {}".format(Y_test.columns[i]))
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

Precision, Recall, F1 score for related
             precision    recall  f1-score   support

          0       0.57      0.38      0.45      1503
          1       0.83      0.91      0.87      4984
          2       0.45      0.45      0.45        42

avg / total       0.77      0.79      0.77      6529

Precision, Recall, F1 score for request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5363
          1       0.85      0.40      0.54      1166

avg / total       0.88      0.88      0.86      6529

Precision, Recall, F1 score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6498
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6529

Precision, Recall, F1 score for aid_related
             precision    recall  f1-score   support

          0       0.70      0.89      0.78      3810
          1       0.75      0.47 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# try Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

#split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

#train pipeline
pipeline.fit(X_train, Y_train)

Y_pred = pipeline.predict(X_test)
col_names = list(Y.columns.values)

for i in range(len(col_names)):
    print("Precision, Recall, F1 score for {}".format(Y_test.columns[i]))
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

Precision, Recall, F1 score for related
             precision    recall  f1-score   support

          0       0.44      0.42      0.43      1503
          1       0.83      0.83      0.83      4984
          2       0.31      0.52      0.39        42

avg / total       0.74      0.74      0.74      6529

Precision, Recall, F1 score for request
             precision    recall  f1-score   support

          0       0.92      0.92      0.92      5418
          1       0.60      0.60      0.60      1111

avg / total       0.86      0.86      0.86      6529

Precision, Recall, F1 score for offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6493
          1       0.04      0.03      0.03        36

avg / total       0.99      0.99      0.99      6529

Precision, Recall, F1 score for aid_related
             precision    recall  f1-score   support

          0       0.74      0.75      0.74      3837
          1       0.63      0.63 

### 9. Export your model as a pickle file

In [15]:
# since RandomForestClassifier performed better, it is selected as the ML model in the study

import pickle

model = cv

pickle.dump(model, open('RFC_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.